In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lish-moa/test_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.0-py3-none-any.whl
/kaggle/input/pytorchtabnet/pytorch_tabnet-1.2.0-py3-none-any.whl
/kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl
/kaggle/input/iterative-stratification/iterative-stratification-master/LICENSE
/kaggle/input/iterative-stratification/iterative-stratification-master/README.md
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.py
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.cfg
/kaggle/input/iterative-stratification/iterative-stratification-master/.gitignore
/kaggle/input/iterative-stratification/iterative-stratification-master/.travis.yml
/kaggle/input/iterative-stratification/iterative-stratification-master/iterstrat/ml_strat

In [2]:
df_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv',index_col='sig_id')
df_test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv',index_col='sig_id')
df_sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv', index_col='sig_id')
df_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv',index_col='sig_id')
df_targets_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv',index_col='sig_id')
df = pd.concat([df_features, df_targets], axis=1)
df_features

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
sig_id,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,trt_cp,24,D1,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,-1.0220,...,0.2862,0.2584,0.8076,0.5523,-0.1912,0.6584,-0.3981,0.2139,0.3801,0.4176
id_000779bfc,trt_cp,72,D1,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,0.2341,...,-0.4265,0.7543,0.4708,0.0230,0.2957,0.4899,0.1522,0.1241,0.6077,0.7371
id_000a6266a,trt_cp,48,D1,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,0.1715,...,-0.7250,-0.6297,0.6103,0.0223,-1.3240,-0.3174,-0.6417,-0.2187,-1.4080,0.6931
id_0015fd391,trt_cp,48,D1,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,-1.9590,...,-2.0990,-0.6441,-5.6300,-1.3780,-0.8632,-1.2880,-1.6210,-0.8784,-0.3876,-0.8154
id_001626bd3,trt_cp,72,D2,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,-0.2800,...,0.0042,0.0048,0.6670,1.0690,0.5523,-0.3031,0.1094,0.2885,-0.3786,0.7125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_fffb1ceed,trt_cp,24,D2,0.1394,-0.0636,-0.1112,-0.5080,-0.4713,0.7201,0.5773,...,0.1969,0.0262,-0.8121,0.3434,0.5372,-0.3246,0.0631,0.9171,0.5258,0.4680
id_fffb70c0c,trt_cp,24,D2,-1.3260,0.3478,-0.3743,0.9905,-0.7178,0.6621,-0.2252,...,0.4286,0.4426,0.0423,-0.3195,-0.8086,-0.9798,-0.2084,-0.1224,-0.2715,0.3689
id_fffc1c3f4,ctl_vehicle,48,D2,0.3942,0.3756,0.3109,-0.7389,0.5505,-0.0159,-0.2541,...,0.5409,0.3755,0.7343,0.2807,0.4116,0.6422,0.2256,0.7592,0.6656,0.3808


In [3]:
def change_df(df):
    df['cp_type'] = df['cp_type'].apply(lambda x: 1 if x == "ctl_vehicle" else 0)
    df['cp_dose'] = df['cp_dose'].apply(lambda x: 1 if x == "D2" else 0)
    df['cp_time'] = df['cp_time'].apply(lambda x: (x - 24)/48)
    return df
df_features = change_df(df_features)
df_test_features = change_df(df_test_features)

In [4]:
df_targets = df_targets[df_features['cp_type'] == 0]
df_targets_nonscored = df_targets_nonscored[df_features['cp_type'] == 0]
df_features = df_features[df_features['cp_type'] == 0]

In [5]:
!pip install --no-index --find-links /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl pytorch-tabnet

Looking in links: /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl
Processing /kaggle/input/pytorchtabnet/pytorch_tabnet-2.0.1-py3-none-any.whl


In [6]:
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from pytorch_tabnet.tab_model import TabNetRegressor
from tqdm import *
from tqdm.keras import TqdmCallback
from imblearn.over_sampling import *

In [7]:
from sklearn.feature_selection import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.decomposition import *
from sklearn.ensemble import *
from sklearn.metrics import *

In [8]:
X = df_features.values
y = df_targets.values
y_nonscored = df_targets_nonscored.values
X_test = df_test_features.values

In [9]:
from scipy import stats
def kstats(X, n):
    return [stats.kstat(x) for x in X]
def kstatvars(X, n):
    return [stats.kstatvar(x) for x in X]
def get_stats(X):
    X_stats_1 = stats.describe(X, axis=1)
    X_quantiles = (np.quantile(X, 0.1, axis=1), np.quantile(X, 0.9, axis=1), np.quantile(X, 0.05, axis=1), np.quantile(X, 0.95, axis=1), np.quantile(X, 0.13, axis=1), np.quantile(X, 0.87, axis=1), np.quantile(X, 0.25, axis=1), np.quantile(X, 0.75, axis=1))
    X_kstats = (kstats(X, 1), kstats(X, 2), kstats(X, 3), kstats(X, 4))
    X_kstatvars = (kstatvars(X, 1), kstatvars(X, 2), kstatvars(X, 3), kstatvars(X, 4))
    X_moments = (stats.moment(X, moment=2, axis=1), stats.moment(X, moment=3, axis=1), stats.moment(X, moment=4, axis=1))
    return (X_stats_1.mean, X_stats_1.variance, X_stats_1.skewness, X_stats_1.kurtosis, X_stats_1.minmax[0], X_stats_1.minmax[1], np.median(X, axis=1), np.sum(X, axis=1), stats.median_absolute_deviation(X, axis=1), stats.variation(X, axis=1)) + X_quantiles + X_kstats + X_kstatvars + X_moments
def get_all_stats(X):
    return get_stats(X[:, 3:-100]) + get_stats(X[:, -100:]) + get_stats(X[:, 3:])

In [10]:
statsselector = VarianceThreshold(threshold=0.1)
statsscaler = MinMaxScaler()

X_stats = np.vstack(get_all_stats(X) + get_all_stats(np.abs(X))).T
#X_stats = statsselector.fit_transform(X_stats)
#X_stats = statsscaler.fit_transform(X_stats)

X_test_stats = np.vstack(get_all_stats(X_test) + get_all_stats(np.abs(X_test))).T
#X_test_stats = statsselector.transform(X_test_stats)
#X_test_stats = statsscaler.transform(X_test_stats)

In [11]:
pca_g = PCA(n_components=100)

X_pca_g = pca_g.fit_transform(X[:, 3:-100])
X_test_pca_g = pca_g.transform(X_test[:, 3:-100])

pca_c = PCA(n_components=20)

X_pca_c = pca_c.fit_transform(X[:, -100:])
X_test_pca_c = pca_c.transform(X_test[:, -100:])


pca_a = PCA(n_components=150)

X_pca_a = pca_a.fit_transform(X[:, 3:])
X_test_pca_a = pca_a.transform(X_test[:, 3:])

pca_s = PCA(n_components=2)

X_pca_s = pca_s.fit_transform(X_stats)
X_test_pca_s = pca_s.transform(X_test_stats)

In [12]:
#selector = VarianceThreshold(threshold=0.5)
#X = np.concatenate([X[:, 1:3], selector.fit_transform(X[:, 3:])], axis=1)
#X_test = np.concatenate([X_test[:, 1:3], selector.transform(X_test[:, 3:])], axis=1)

In [13]:
skiptransform = 3
for i in tqdm(range(X.shape[1]-skiptransform)):
    scaler = QuantileTransformer(output_distribution="normal")
    X[:, i+skiptransform] = scaler.fit_transform(X[:, i+skiptransform].reshape(-1, 1)).reshape(-1)
    X_test[:, i+skiptransform] = scaler.transform(X_test[:, i+skiptransform].reshape(-1, 1)).reshape(-1)

100%|██████████| 872/872 [00:10<00:00, 80.34it/s]


In [14]:
def get_stats_names():
    X_quantiles = ("q0.1", "q0.9", "q0.05", "q0.95", "q0.13", "q0.87", "q0.25", "q0.75")
    X_kstats = ("ks1", "ks2", "ks3", "ks4")
    X_kstatvars = ("ksv1", "ksv2", "ksv3", "ksv4")
    X_moments = ("moment2", "moment3", "moment4")
    return ("mean", "variance", "skewness", "kurtosis", "min", "max", "median", "sum", "medianabsdev", "var") + X_quantiles + X_kstats + X_kstatvars + X_moments
def get_all_stats_names():
    return ["g_" + x for x in get_stats_names()] + ["c_" + x for x in get_stats_names()] + ["a_" + x for x in get_stats_names()]
def get_stat_names():
    return get_all_stats_names() + ["abs_" + x for x in get_all_stats_names()]

In [15]:
#X.shape
#feature_names = list(map(str,range(875))) + get_stat_names() + ["pcag_" + str(i) for i in range(100)] + ["pcac_" + str(i) for i in range(20)] + ["pcaa_" + str(i) for i in range(150)] + ["pcas_" + str(i) for i in range(2)]\#import seaborn as sns
#feature_importances_df = pd.DataFrame({"features":feature_names, "importances":importances}).sort_values(by=["importances"],axis=0,ascending=False)
#plt.figure(figsize=(16,256))
#sns.barplot(x="importances",y="features",data=feature_importances_df)

In [16]:
X = np.concatenate([X, X_stats, X_pca_g, X_pca_c, X_pca_a, X_pca_s], axis=1)
X_test = np.concatenate([X_test, X_test_stats, X_test_pca_g, X_test_pca_c, X_test_pca_a, X_test_pca_s], axis=1)

In [17]:
importances = [0.0,0.014828467,0.020225618,0.2016071,0.12000491,0.23457073,0.14336859,0.188986,0.15212588,0.06948359,0.087405145,0.23941657,0.15326,0.123934634,0.12399402,0.103303656,0.20058796,0.12029886,0.17182171,0.17401007,0.13714683,0.12812747,0.105611466,0.1362481,0.21370016,0.2196767,0.17299509,0.11271816,0.16150045,0.08740312,0.17659304,0.24062175,0.14534806,0.2081732,0.21428908,0.08611817,0.13701023,0.13773647,0.26561368,0.1434312,0.10359177,0.11360872,0.39624667,0.15491325,0.15722564,0.09098127,0.09633679,0.16126265,0.10018399,0.15032308,0.23942244,0.22126837,0.28376353,0.1888057,0.17380139,0.17511089,0.115856364,0.1279153,0.11989386,0.18481423,0.17528455,0.20045982,0.08640442,0.14467753,0.21244904,0.19943722,0.14487082,0.22226937,0.12023558,0.16604203,0.0930883,0.27884412,0.2029998,0.21041453,0.08665657,0.1625424,0.17050163,0.23688473,0.2311762,0.11819308,0.08994279,0.107802816,0.08963323,0.072145164,0.16317935,0.18093807,0.16377741,0.12995249,0.20732547,0.20985179,0.09297665,0.09443255,0.13126844,0.16180238,0.35355374,0.1406772,0.11240736,0.123544924,0.21902955,0.24269499,0.3128133,0.15925594,0.17834134,0.255089,0.114810154,0.16059443,0.1341405,0.17533413,0.16721605,0.28744242,0.094345815,0.20214863,0.10101388,0.13952431,0.15736425,0.17649394,0.24808577,0.22200182,0.13782644,0.13008747,0.13837633,0.14264506,0.10602203,0.28742203,0.15291165,0.10039946,0.19622512,0.1389252,0.20876105,0.2653083,0.13136384,0.1488323,0.22270612,0.20330262,0.12919706,0.21354243,0.13762994,0.16603364,0.23220961,0.14787713,0.210397,0.18417796,0.10112901,0.20152037,0.1316053,0.19314227,0.12204596,0.23277669,0.1699048,0.18309446,0.25057808,0.23095118,0.15028441,0.19247042,0.14718923,0.20089255,0.13703424,0.19826141,0.16269271,0.16793756,0.26546153,0.38431594,0.15324688,0.14877754,0.2553587,0.17273922,0.22963563,0.1402346,0.12930073,0.13973291,0.13163508,0.17079353,0.17546055,0.33631897,0.14606075,0.16968909,0.14460301,0.16917604,0.29222792,0.15800907,0.1817757,0.23499127,0.13962394,0.17095338,0.1758315,0.12613574,0.19751099,0.058083136,0.1713235,0.18966784,0.13723525,0.14090547,0.20406362,0.17361549,0.13757901,0.34818873,0.10061631,0.16109128,0.092076406,0.2122965,0.1513452,0.115156904,0.12801008,0.12353009,0.2513353,0.25009686,0.25588995,0.25987092,0.11569356,0.20323749,0.1342574,0.2194858,0.14551783,0.14683756,0.253608,0.13107216,0.16325064,0.16157909,0.11793301,0.1148031,0.09843255,0.09724129,0.15283266,0.21329463,0.11516629,0.15879339,0.16150844,0.06949459,0.07454359,0.18079036,0.06193443,0.22941495,0.17516415,0.092276745,0.2414239,0.12694673,0.1533688,0.18261592,0.23765421,0.124658935,0.110062756,0.11309241,0.15654184,0.19229764,0.086832866,0.1352812,0.10497989,0.19109577,0.15798284,0.09815823,0.21209009,0.21001738,0.23912439,0.30291554,0.092174605,0.15245406,0.21402375,0.24776834,0.20405102,0.11291308,0.21251707,0.12861124,0.29730424,0.16609295,0.21818733,0.14118303,0.19291966,0.17233095,0.106412515,0.15411262,0.22597788,0.15102787,0.15689257,0.23826618,0.14925279,0.20910735,0.09837827,0.18639366,0.11170214,0.20677245,0.099646874,0.17598413,0.09124955,0.15147772,0.13200833,0.17003085,0.14380157,0.14345306,0.11884663,0.15469645,0.09662017,0.13128102,0.09842358,0.17879178,0.13867247,0.13497113,0.14510687,0.1554322,0.20488067,0.22105068,0.18028775,0.18566984,0.13360126,0.19736166,0.17691706,0.19545779,0.103025466,0.19464184,0.13557518,0.14531209,0.10245004,0.1876193,0.15959914,0.24287337,0.17703366,0.16494542,0.19042529,0.07692109,0.1549392,0.1446259,0.2677974,0.16872093,0.11168945,0.30778867,0.11887219,0.15321176,0.12574941,0.22066799,0.115919665,0.13435091,0.13384666,0.12369856,0.15411334,0.1539326,0.11912916,0.17390627,0.18102586,0.10841074,0.19988114,0.11104967,0.109710105,0.13444549,0.1746575,0.09808462,0.15849285,0.17797863,0.13417973,0.23139599,0.08651519,0.18651316,0.10530115,0.124492675,0.14829114,0.1670182,0.18935975,0.22670054,0.2048896,0.18635212,0.15922032,0.09970511,0.19334313,0.116435654,0.09318605,0.16666977,0.28689903,0.1866297,0.13902554,0.21502726,0.18333495,0.14027192,0.17459817,0.12575492,0.18351899,0.17956336,0.06564469,0.21264315,0.1467276,0.1977767,0.13845101,0.15132341,0.26086068,0.07616704,0.23642363,0.12989958,0.18171726,0.1971073,0.12176068,0.10340516,0.2538998,0.28393194,0.22091302,0.18852638,0.08043113,0.22312981,0.14104785,0.5002081,0.11584934,0.17273608,0.14733237,0.16118,0.06657026,0.1823547,0.1568675,0.16101837,0.12697,0.19904828,0.14515416,0.13960925,0.24750869,0.21008514,0.23772092,0.08053175,0.23447709,0.23450477,0.13559815,0.15888993,0.12995887,0.19434524,0.09370749,0.16411103,0.1609322,0.21847342,0.15166703,0.13534077,0.08130669,0.17621891,0.10745025,0.14751014,0.09585203,0.17146987,0.15984112,0.21148555,0.12667306,0.32787496,0.19183326,0.10539532,0.37064648,0.19043973,0.13953455,0.2854501,0.15974028,0.13152872,0.18960083,0.10401599,0.20302165,0.11141246,0.20603858,0.06859658,0.17935105,0.15003976,0.17535444,0.056019578,0.24101198,0.17196082,0.12861514,0.14547418,0.13772741,0.14900719,0.13913527,0.15139844,0.15018487,0.07099197,0.16819073,0.18126975,0.16402765,0.1601871,0.20045336,0.2269576,0.2251456,0.19212224,0.13110605,0.22659846,0.13294725,0.20131627,0.10892054,0.13183205,0.11084218,0.20371333,0.12783645,0.08853352,0.21168002,0.1871751,0.16751686,0.10244215,0.14926414,0.10828553,0.22879444,0.15969826,0.102821164,0.14161517,0.09925784,0.12017577,0.20692366,0.20554039,0.14908269,0.18313076,0.12099518,0.15764755,0.120397694,0.18379338,0.09269632,0.11506224,0.13628791,0.18370822,0.17215121,0.12853698,0.19150284,0.10367365,0.18432595,0.22583187,0.11999348,0.09427908,0.12354278,0.1906542,0.1425651,0.23157692,0.23365246,0.16424042,0.16776595,0.16477978,0.107901275,0.20132771,0.21062377,0.09598201,0.19090162,0.17025048,0.11380147,0.11528879,0.14708665,0.19838779,0.12122913,0.20214196,0.12757908,0.12520199,0.2535199,0.16358209,0.1147228,0.14861088,0.23388602,0.2091873,0.13453206,0.24071428,0.11378684,0.06563051,0.13662401,0.10903789,0.10801051,0.10925899,0.10380262,0.2234169,0.10461396,0.14907792,0.10787836,0.110770114,0.11928711,0.13440405,0.15983798,0.1593943,0.1997959,0.08721618,0.20371252,0.21250106,0.14177248,0.060442764,0.11035692,0.12943707,0.19835691,0.12368999,0.10770731,0.121647514,0.10607326,0.23051392,0.22565673,0.16049227,0.19668871,0.16713265,0.19517678,0.25655374,0.11209349,0.1984877,0.12996912,0.14567001,0.13111666,0.114025526,0.11641072,0.17421505,0.09661984,0.15027225,0.123419516,0.1029895,0.09362584,0.08465493,0.14438933,0.15494238,0.12581106,0.15031834,0.20062293,0.1282448,0.15997796,0.16865115,0.16561007,0.1629409,0.09266446,0.2676025,0.17701437,0.11075198,0.10598207,0.19154614,0.11726609,0.18728745,0.1517574,0.0990039,0.08155019,0.2051116,0.16457961,0.11800892,0.09240267,0.17735845,0.11463352,0.09936446,0.10443141,0.14717942,0.18550082,0.21976107,0.108400635,0.053466592,0.22192302,0.18196754,0.15456522,0.23112859,0.13397029,0.2997361,0.13465245,0.12057249,0.11004971,0.13333714,0.09277951,0.1934592,0.16127068,0.14023662,0.2176794,0.19067134,0.39564723,0.1976939,0.15029673,0.13697076,0.22966279,0.17530942,0.10749717,0.14212915,0.21912634,0.18673415,0.15085481,0.13937625,0.12976593,0.14236768,0.18888085,0.22437131,0.12168428,0.24755609,0.11656674,0.077900544,0.071262725,0.20980997,0.1986534,0.14913078,0.09867303,0.14049672,0.18936162,0.17493597,0.22104774,0.15140447,0.11098374,0.24963689,0.15217026,0.10863851,0.10494942,0.10451082,0.18001261,0.16567379,0.23170233,0.13922171,0.1467623,0.18115786,0.18382588,0.09606831,0.12861286,0.20883627,0.07644925,0.17933011,0.09429676,0.16993833,0.14660859,0.09341625,0.13745992,0.10997003,0.18462977,0.123799436,0.21261887,0.22891036,0.06980604,0.10206877,0.15071829,0.20572375,0.20633188,0.18038246,0.21347858,0.19372335,0.19281814,0.10633706,0.25249052,0.29590234,0.12686096,0.17500745,0.116756245,0.16791861,0.18448114,0.20177992,0.15757377,0.12422465,0.16367309,0.25207293,0.20317999,0.16714664,0.20012285,0.13475348,0.24689028,0.15148918,0.11968315,0.23033242,0.1369771,0.19423167,0.19783272,0.11771795,0.26564828,0.20418105,0.07985115,0.10739124,0.19816911,0.15884267,0.19523078,0.08898052,0.21458463,0.13412172,0.11734435,0.15460914,0.1922472,0.22758591,0.15779284,0.10150323,0.18417135,0.11971457,0.11446574,0.12834762,0.15777072,0.16835514,0.07511905,0.13005875,0.12782326,0.13317214,0.16424967,0.12200498,0.19839695,0.16397095,0.28319702,0.09375124,0.22822934,0.19456734,0.24728212,0.11484218,0.15351853,0.09550887,0.118261516,0.08495805,0.14068285,0.14947017,0.16256264,0.17164965,0.17704017,0.12073288,0.20783909,0.2103546,0.17786404,0.20312963,0.329774,0.16134575,0.10296745,0.18679507,0.1566408,0.20145981,0.17542382,0.21648736,0.16284172,0.12375012,0.31855085,0.20934889,0.20528194,0.15570948,0.14095902,0.16875288,0.14542598,0.15512386,0.20911306,0.12828754,0.148122,0.14526999,0.14272563,0.13148448,0.19980319,0.1561675,0.15456744,0.23261973,0.14042881,0.1766904,0.1908503,0.1057508,0.12731372,0.11082747,0.13525952,0.12732863,0.15921879,0.15775086,0.178228,0.25063777,0.15005273,0.25941613,0.18393631,0.29294893,0.08620091,0.15310144,0.14268944,0.14526911,0.23011135,0.17844038,0.1698354,0.107403174,0.17790654,0.1953118,0.2209334,0.12310632,0.23974861,0.074262515,0.26975212,0.16272439,0.1734638,0.17659187,0.13353173,0.14372893,0.18955371,0.13982607,0.17816299,0.16330308,0.16209127,0.39274675,0.20741683,0.15868585,0.18592295,0.21463206,0.15746628,0.13297635,0.12691876,0.25795272,0.12509504,0.13545674,0.14947978,0.16548319,0.19791149,0.14846934,0.23792449,0.16846089,0.17126028,0.15938523,0.11429151,0.26783442,0.14114873,0.08762761,0.0,0.21428968,0.12565997,0.11737516,0.048088428,0.098975375,0.003426692,0.17489426,0.057404537,0.11601403,0.20326573,0.19665049,0.21508244,0.14078523,0.24737689,0.34087783,0.2864603,9.7609925e-05,0.0,0.0,0.39724463,0.18877877,0.00074617367,0.0,0.0,0.0,0.10993121,0.16699691,0.027600557,0.24962647,0.1942778,0.118214,0.14940529,0.10450875,0.05071179,0.0,0.2120705,0.21857448,0.22710773,0.026176853,0.15681386,0.0566556,0.06321438,0.020579776,0.036198746,0.024583774,0.018344926,0.0,0.0,0.0,0.19641048,0.0,0.0,0.0,0.00041641662,0.16379735,0.13504355,0.05983236,0.31618345,0.24894427,0.10410841,0.2543099,0.26873627,0.05718469,0.0,0.14528489,0.14193945,0.17635885,0.3247789,0.22635384,0.26182917,0.100433215,0.22045228,0.15462676,0.4536346,0.004405347,0.0,0.0,0.0,0.07903533,0.0,0.0,0.0038050322,0.0,0.096362405,0.18776715,0.2276517,0.21211185,0.14634816,0.17533424,0.0067027053,0.102751605,0.117233254,0.0,0.3326634,0.13132171,0.28490946,0.19465008,0.2681391,0.32042074,0.24218775,0.13305306,0.14137423,0.17766121,0.0070629395,0.0,0.0,0.0,0.15696232,0.0,0.0006860478,0.0,0.0,0.05452498,0.012388085,0.10797828,0.14612146,0.24120165,0.113694996,0.17698313,0.15904614,0.18265171,0.0,0.16642646,0.14242871,0.18684681,0.18612844,0.18479331,0.25339666,0.1864008,0.2686271,0.22194247,0.14342473,0.00019998514,0.0,0.0,0.0,0.23459561,0.00058014854,0.0,0.0,0.0,0.07279152,0.18223839,0.00506893,0.0,0.10796081,0.12652378,0.015261299,0.28564796,0.1859774,0.08619577,0.2756688,0.15325835,0.33570385,0.19271302,0.2763128,0.32123342,0.17412603,0.20905906,0.14185156,0.22849531,0.45261168,0.0,0.0,0.0,0.13209811,0.0,0.0,0.0,0.0,0.20217778,0.10953993,0.19171183,0.16141324,0.018865308,0.063189805,0.09060589,0.15825345,0.07064646,0.04161437,0.09782687,0.077117406,0.10935104,0.14314777,0.27479357,0.10058603,0.07912111,0.13031605,0.2323272,0.15886974,0.15586567,0.105438076,0.08757085,0.29638714,0.084067225,0.10342219,0.2181943,0.12793078,0.08236317,0.085319705,0.18215537,0.17644529,0.08573001,0.2168619,0.11743749,0.08026688,0.17251727,0.14962086,0.11688927,0.1062106,0.15970078,0.18990704,0.17013876,0.13333222,0.13731359,0.17551357,0.08118564,0.09854721,0.121522695,0.15408084,0.22952995,0.10259158,0.13281639,0.09476331,0.13190027,0.16879871,0.06399906,0.2210667,0.11574354,0.16566049,0.116357245,0.20235206,0.15236089,0.14121805,0.124011666,0.08236247,0.19930927,0.13526879,0.13275562,0.22472371,0.12160813,0.18089278,0.13637784,0.085714474,0.16880819,0.22781835,0.15607242,0.14225616,0.14391194,0.07480363,0.14495905,0.14021347,0.13087635,0.15832251,0.10436686,0.23197944,0.13770266,0.2082517,0.12036577,0.15517356,0.20270811,0.21111833,0.13566111,0.28519273,0.10252782,0.14549787,0.19786313,0.103967965,0.11212763,0.20374005,0.20373197,0.20785107,0.011708892,0.9069087,0.21095513,0.29592314,0.37165985,0.22700973,0.22170475,0.2512499,0.28131196,0.1860217,0.18186261,0.24854831,0.16793235,0.20685916,0.16276252,0.22882433,0.2805822,0.22784497,0.17137878,0.2473776,0.15570365,0.12808716,0.14200547,0.09234593,0.12513824,0.10653392,0.020417621,0.041773796,0.158524,0.22455755,0.105593994,0.16408567,0.20812678,0.10248729,0.22143584,0.14980902,0.22584899,0.11205671,0.23051113,0.20196168,0.12661612,0.093336664,0.20644012,0.30560282,0.13965806,0.14134084,0.2045498,0.16982979,0.16499168,0.16057058,0.10376714,0.10985822,0.10037088,0.13636836,0.120936505,0.1695152,0.13566665,0.15371986,0.09951323,0.14163925,0.12577142,0.12385576,0.16153261,0.18676373,0.15000209,0.12037751,0.13303758,0.18142731,0.117437385,0.1431659,0.21746822,0.09016984,0.13409004,0.151535,0.13027512,0.13042188,0.15884422,0.11096528,0.14724149,0.15739907,0.13981366,0.15715231,0.1307134,0.13490745,0.19710706,0.1916286,0.19271377,0.16446048,0.10896732,0.1788321,0.11769756,0.19572386,0.113424644,0.096478954,0.10657286,0.20802866,0.24059504,0.20759884,0.12265568,0.16695799,0.16989341,0.15445739,0.17647953,0.20451002,0.11226781,0.15948148,0.14618644,0.21779494,0.16918138,0.1858859,0.11841095,0.14387324,0.12845936,0.17614594,0.108967334,0.21842116,0.13213934,0.1078793,0.13312607,0.17003977,0.12169365,0.14017668,0.1808543,0.09276705,0.09032219,0.09113362,0.13909952,0.12020604,0.16157186,0.15711677,0.12767687,0.22417182,0.17937702,0.17269225,0.13078393,0.1611634,0.16726302,0.1435551,0.12480039,0.19917755,0.12406161,0.14129983,0.20955378,0.13029741,0.107799806,0.09242775,0.12634046,0.14624588,0.11843404,0.18360682,0.15005288,0.08842365,0.12042121,0.097905055,0.09159432,0.17700553,0.09512616,0.12057748,0.14641461,0.1605967,0.124667935,0.095523134,0.17615896,0.10764849,0.11766861,0.21454982,0.16646287,0.19806334,0.14190806,0.14686048,0.20713489,0.19923756]

In [18]:
feature_select = [1150,395,950,1038,896,42,642,853,161,1153,436,94,195,891,173,1030,970,784,433,944,1033,975,794,934,100,323,1192,253,631,262,1070,1152,711,827,178,109,123,364,892,1025,439,1140,972,389,52,764,1157,1165,71,1032,1028,1061,842,938,1006,974,873,320,603,734,38,160,129,946,380,207,825,861,577,206,164,103,937,388,214,534,1004,710,721,204,1156,823,150,205,673,905,935,1160,116,257,659,408,1168,890,768,726,313,99,976,234,993,452,541,31,1245,840,50,11,252,273,868,410,238,77,382,181,1013,5,413,412,538,516,147,811,1065,138,1132,680,515,347,78,629,151,571,1187,729,832,646,166,1097,231,699,1164,486,1037,766,1166,1122,962,747,914,1154,467,355,471,945,270,1185,509,572,468,1116,1178,657,1280,549,393,132,67,117,1007,626,1155,1183,51,1104,299,670,838,390,327,948,623,25,211,650,98,913,421,1264,1073,264,1256,640,1219,1080,791,888,367,857,742,1314,877,34,256,24,135,706,223,375,698,260,561,64,199,963,250,912,480,431,1138,1151,522,73,140,781,409,251,89,663,1291,795,539,802,275,1035,687,128,1134,33,1181,1244,1148,780,1246,854,88,1319,492,1162,279,1191,704,446,703,493,796,613,356,298,1195,1252,735,192,258,1146,1147,477,560,133,886,209,722,783,444,72,1137,1108,1047,111,531,1188,717,3,143,789,521,473,155,596,16,61,466,724,338,808,558,65,1113,1320,1288,405,664,579,762,529,566,157,738,1316,866,1143,732,377,643,186,303,385,1233,574,887,926,126,1240,305,837,740,576,973,307,767,417,906,731,707,637,360,145,266,708,1235,1031,153,243,746,469,434,1049,1234,607,506,247,524,814,641,513,437,316,1088,189,442,848,668,354,7,656,53,897,391,961,311,609,481,1001,787,1212,651,365,349,1005,277,357,1002,1158,1026,856,1258,301,622,59,1003,696,716,508,141,750,826,684,499,503,1298,372,368,495,149,997,237,401,1019,1077,627,1159,180,384,1216,463,683,336,85,1118,1271,229,705,300,678,373,1281,448,689,1238,293,833,102,822,850,345,836,782,979,617,778,314,604,1304,995,304,813,30,845,115,1251,1078,943,425,1311,1262,281,184,1092,172,790,450,965,107,647,60,232,55,713,669,883,342,370,585,1034,19,335,54,193,844,26,165,397,1282,1083,267,504,453,18,777,429,1167,188,870,183,171,76,525,1089,1268,285,691,148,1249,834,1196,175,1204,1257,177,1121,1102,799,321,599,869,755,462,159,1161,715,518,482,1285,108,723,575,353,903,1248,363,1315,999,263,69,137,679,1106,600,865,1197,315,519,614,1236,760,517,419,1180,464,763,931,86,720,535,851,216,84,601,792,1163,843,158,776,75,852,93,217,1277,1211,226,28,1050,785,638,47,399,1284,197,403,420,1308,105,1198,573,465,598,430,556,440,1087,487,312,1254,557,871,101,358,820,996,415,1066,1225,739,225,855,1177,344,1130,1054,179,248,748,754,821,497,718,858,1228,114,44,1230,1278,984,272,402,916,788,242,809,1123,1067,797,1169,297,1136,801,593,318,43,289,949,745,810,628,1250,332,269,1096,333,1206,770,236,12,1029,162,325,829,124,222,255,1109,674,8,610,422,1222,727,283,671,459,200,379,271,652,702,49,595,644,152,587,460,1299,824,449,1213,1184,1084,864,775,908,484,274,665,494,551,457,131,163,537,867,352,804,139,427,398,1227,154,621,528,1318,213,682,376,692,1307,964,1296,1255,992,174,581,212,1142,455,800,32,309,941,805,831,406,296,1127,66,63,319,176,592,1125,1260,286,847,1286,287,39,1008,6,1218,1060,806,830,121,514,1000,655,1124,649,1171,942,1317,1036,562,1208,489,978,1194,1290,1110,265,874,394,798,191,889,774,95,667,812,369,639,167,1128,1270,849,1229,169,1193,182,407,438,113,653,681,458,1275,366,127,294,378,120,118,37,456,1133,136,194,694,1091,190,20,156,36,730,645,544,1119,1202,502,23,1205,1139,414,308,863,423,245,1114,818,932,295,1232,725,632,540,341,555,329,210,346,106,743,1221,630,330,302,846,635,1090,759,1267,977,1215,859,472,1099,1115,1265,1042,284,1101,475,170,144,441,807,130,971,291,92,582,470,215,1129,1283,1231,1224,1064,1292,1223,119,757,580,416,87,383,654,565,168,134,454,686,261,505,1261,753,803,597,21,1170,202,1074,57,478,758,1279,532,819,816,404,235,860,712,432,1189,1023,1295,185,594,1209,371,326,878,533,1173,862,1287,1309,239,351,719,1289,1111,14,13,1210,697,793,331,567,97,512,203,588,839,1247,146,761,386,1269,658,569,1117,1095,530,496,1203,779,1306,633,1301,498,1214,1135,17,68,1276,491,4,510,58,751,728,554,334,324,288,1297,1259,772,907,79,615,218,733,1239,1313,1081,1217,879,744,608,968,1085,714,660,361,584,1107,885,328,56,396,1105,208,527,224,201,501,769,104,219,536,618,752,872,583,526,542,994,41,1241,241,259,27,96,1253,1145,578,1186,278,322,445,339,672,1226,476,817,553,605,564,240,634,695,902,1200,340,1048,1059,547,545,1263,1237,474,675,337,624,485,546,991,1022,520,1266,552,81,1293,568,1312,648,426,835,737,1243,1174,268,709,1086,570,122,606,815,22,1179,1068,435,350,246,676,550,677,909,620,1131,936,443,1144,548,1199,507,40,1072,387,15,306,589,786,488,967,1098,1141,1182,310,483,701,749,142,112,196,1062,947,125,1201,48,359,280,1207,619,490,611,881,666,1094,220,292,276,249,343,1302,1057,221,290,586,1242,960,46,685,523,428,1310,771,1305,1100,91,110,690,511,765,418,590,693,1190,362,70,90,636,1272,500,602,1294,616,1172,233,254,198,1303,282,1274,45,1053,1273,1220,80,82,741,479,1300,875,1069,10,29,559,244,74,348,62,828,1027,35,1079,1120,1076,773,591,1071,1075,1112,612,424,1093,411,392,1082,736,1063,955,661,1058,317,688,381,756,1126,228,841,1018,83,662,461,1055,700,227,9,447,400,374,543,1103,918,1052,230,563,933,187,884,939,917,451,989,625,910,880,1176,1056,920,904,915,921,919,1175,2,1051,922,1024,1,990,1149,980,966,1020,951,958,882,898,986,1014,930,1009,893,923,924,900,911,901,927,899,895,894,876,925,952,928,1010,1046,1045,1044,1043,1041,1040,1039,1021,1017,1016,1015,1012,1011,998,929,988,987,985,983,982,981,969,959,957,956,954,953,940,0]
feature_select = feature_select[:900]
X = X[:, feature_select]
X_test = X_test[:, feature_select]

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


device = ('cuda' if torch.cuda.is_available() else 'cpu')

if device == 'cpu':
    torch.set_num_threads(4)

In [20]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float).to(device)
        self.targets = torch.tensor(targets, dtype=torch.float).to(device)
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

class MoATestDataset:
    def __init__(self, features):
        self.features = torch.tensor(features, dtype=torch.float).to(device)
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        return self.features[idx]

In [21]:
# Lookahead implementation from https://github.com/rwightman/pytorch-image-models/blob/master/timm/optim/lookahead.py

""" Lookahead Optimizer Wrapper.
Implementation modified from: https://github.com/alphadl/lookahead.pytorch
Paper: `Lookahead Optimizer: k steps forward, 1 step back` - https://arxiv.org/abs/1907.08610
"""
import torch
from torch.optim.optimizer import Optimizer
from torch.optim import Adam
from collections import defaultdict

class Lookahead(Optimizer):
    def __init__(self, base_optimizer, alpha=0.5, k=6):
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        defaults = dict(lookahead_alpha=alpha, lookahead_k=k, lookahead_step=0)
        self.base_optimizer = base_optimizer
        self.param_groups = self.base_optimizer.param_groups
        self.defaults = base_optimizer.defaults
        self.defaults.update(defaults)
        self.state = defaultdict(dict)
        # manually add our defaults to the param groups
        for name, default in defaults.items():
            for group in self.param_groups:
                group.setdefault(name, default)

    def update_slow(self, group):
        for fast_p in group["params"]:
            if fast_p.grad is None:
                continue
            param_state = self.state[fast_p]
            if 'slow_buffer' not in param_state:
                param_state['slow_buffer'] = torch.empty_like(fast_p.data)
                param_state['slow_buffer'].copy_(fast_p.data)
            slow = param_state['slow_buffer']
            slow.add_(group['lookahead_alpha'], fast_p.data - slow)
            fast_p.data.copy_(slow)

    def sync_lookahead(self):
        for group in self.param_groups:
            self.update_slow(group)

    def step(self, closure=None):
        # print(self.k)
        #assert id(self.param_groups) == id(self.base_optimizer.param_groups)
        loss = self.base_optimizer.step(closure)
        for group in self.param_groups:
            group['lookahead_step'] += 1
            if group['lookahead_step'] % group['lookahead_k'] == 0:
                self.update_slow(group)
        return loss

    def state_dict(self):
        fast_state_dict = self.base_optimizer.state_dict()
        slow_state = {
            (id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict['state']
        param_groups = fast_state_dict['param_groups']
        return {
            'state': fast_state,
            'slow_state': slow_state,
            'param_groups': param_groups,
        }

    def load_state_dict(self, state_dict):
        fast_state_dict = {
            'state': state_dict['state'],
            'param_groups': state_dict['param_groups'],
        }
        self.base_optimizer.load_state_dict(fast_state_dict)

        # We want to restore the slow state, but share param_groups reference
        # with base_optimizer. This is a bit redundant but least code
        slow_state_new = False
        if 'slow_state' not in state_dict:
            print('Loading state_dict from optimizer without Lookahead applied.')
            state_dict['slow_state'] = defaultdict(dict)
            slow_state_new = True
        slow_state_dict = {
            'state': state_dict['slow_state'],
            'param_groups': state_dict['param_groups'],  # this is pointless but saves code
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.param_groups = self.base_optimizer.param_groups  # make both ref same container
        if slow_state_new:
            # reapply defaults to catch missing lookahead specific ones
            for name, default in self.defaults.items():
                for group in self.param_groups:
                    group.setdefault(name, default)

def LookaheadAdam(params, alpha=0.5, k=6, *args, **kwargs):
     adam = Adam(params, *args, **kwargs)
     return Lookahead(adam, alpha, k)

In [22]:


import contextlib
from torch.autograd import Function
from collections import OrderedDict
from torch.jit import script
from warnings import warn
import os
import glob
import hashlib
import gc
import time
import requests

def to_one_hot(y, depth=None):
    r"""
    Takes integer with n dims and converts it to 1-hot representation with n + 1 dims.
    The n+1'st dimension will have zeros everywhere but at y'th index, where it will be equal to 1.
    Args:
        y: input integer (IntTensor, LongTensor or Variable) of any shape
        depth (int):  the size of the one hot dimension
    """
    y_flat = y.to(torch.int64).view(-1, 1)
    depth = depth if depth is not None else int(torch.max(y_flat)) + 1
    y_one_hot = torch.zeros(y_flat.size()[0], depth, device=y.device).scatter_(1, y_flat, 1)
    y_one_hot = y_one_hot.view(*(tuple(y.shape) + (-1,)))
    return y_one_hot


def _make_ix_like(input, dim=0):
    d = input.size(dim)
    rho = torch.arange(1, d + 1, device=input.device, dtype=input.dtype)
    view = [1] * input.dim()
    view[0] = -1
    return rho.view(view).transpose(0, dim)


class SparsemaxFunction(Function):
    """
    An implementation of sparsemax (Martins & Astudillo, 2016). See
    :cite:`DBLP:journals/corr/MartinsA16` for detailed description.
    By Ben Peters and Vlad Niculae
    """

    @staticmethod
    def forward(ctx, input, dim=-1):
        """sparsemax: normalizing sparse transform (a la softmax)
        Parameters:
            input (Tensor): any shape
            dim: dimension along which to apply sparsemax
        Returns:
            output (Tensor): same shape as input
        """
        ctx.dim = dim
        max_val, _ = input.max(dim=dim, keepdim=True)
        input -= max_val  # same numerical stability trick as for softmax
        tau, supp_size = SparsemaxFunction._threshold_and_support(input, dim=dim)
        output = torch.clamp(input - tau, min=0)
        ctx.save_for_backward(supp_size, output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        supp_size, output = ctx.saved_tensors
        dim = ctx.dim
        grad_input = grad_output.clone()
        grad_input[output == 0] = 0

        v_hat = grad_input.sum(dim=dim) / supp_size.to(output.dtype).squeeze()
        v_hat = v_hat.unsqueeze(dim)
        grad_input = torch.where(output != 0, grad_input - v_hat, grad_input)
        return grad_input, None


    @staticmethod
    def _threshold_and_support(input, dim=-1):
        """Sparsemax building block: compute the threshold
        Args:
            input: any dimension
            dim: dimension along which to apply the sparsemax
        Returns:
            the threshold value
        """

        input_srt, _ = torch.sort(input, descending=True, dim=dim)
        input_cumsum = input_srt.cumsum(dim) - 1
        rhos = _make_ix_like(input, dim)
        support = rhos * input_srt > input_cumsum

        support_size = support.sum(dim=dim).unsqueeze(dim)
        tau = input_cumsum.gather(dim, support_size - 1)
        tau /= support_size.to(input.dtype)
        return tau, support_size


sparsemax = lambda input, dim=-1: SparsemaxFunction.apply(input, dim)
sparsemoid = lambda input: (0.5 * input + 0.5).clamp_(0, 1)


class Entmax15Function(Function):
    """
    An implementation of exact Entmax with alpha=1.5 (B. Peters, V. Niculae, A. Martins). See
    :cite:`https://arxiv.org/abs/1905.05702 for detailed description.
    Source: https://github.com/deep-spin/entmax
    """

    @staticmethod
    def forward(ctx, input, dim=-1):
        ctx.dim = dim

        max_val, _ = input.max(dim=dim, keepdim=True)
        input = input - max_val  # same numerical stability trick as for softmax
        input = input / 2  # divide by 2 to solve actual Entmax

        tau_star, _ = Entmax15Function._threshold_and_support(input, dim)
        output = torch.clamp(input - tau_star, min=0) ** 2
        ctx.save_for_backward(output)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        Y, = ctx.saved_tensors
        gppr = Y.sqrt()  # = 1 / g'' (Y)
        dX = grad_output * gppr
        q = dX.sum(ctx.dim) / gppr.sum(ctx.dim)
        q = q.unsqueeze(ctx.dim)
        dX -= q * gppr
        return dX, None

    @staticmethod
    def _threshold_and_support(input, dim=-1):
        Xsrt, _ = torch.sort(input, descending=True, dim=dim)

        rho = _make_ix_like(input, dim)
        mean = Xsrt.cumsum(dim) / rho
        mean_sq = (Xsrt ** 2).cumsum(dim) / rho
        ss = rho * (mean_sq - mean ** 2)
        delta = (1 - ss) / rho

        # NOTE this is not exactly the same as in reference algo
        # Fortunately it seems the clamped values never wrongly
        # get selected by tau <= sorted_z. Prove this!
        delta_nz = torch.clamp(delta, 0)
        tau = mean - torch.sqrt(delta_nz)

        support_size = (tau <= Xsrt).sum(dim).unsqueeze(dim)
        tau_star = tau.gather(dim, support_size - 1)
        return tau_star, support_size


class Entmoid15(Function):
    """ A highly optimized equivalent of labda x: Entmax15([x, 0]) """

    @staticmethod
    def forward(ctx, input):
        output = Entmoid15._forward(input)
        ctx.save_for_backward(output)
        return output

    @staticmethod
    @script
    def _forward(input):
        input, is_pos = abs(input), input >= 0
        tau = (input + torch.sqrt(F.relu(8 - input ** 2))) / 2
        tau.masked_fill_(tau <= input, 2.0)
        y_neg = 0.25 * F.relu(tau - input, inplace=True) ** 2
        return torch.where(is_pos, 1 - y_neg, y_neg)

    @staticmethod
    def backward(ctx, grad_output):
        return Entmoid15._backward(ctx.saved_tensors[0], grad_output)

    @staticmethod
    @script
    def _backward(output, grad_output):
        gppr0, gppr1 = output.sqrt(), (1 - output).sqrt()
        grad_input = grad_output * gppr0
        q = grad_input / (gppr0 + gppr1)
        grad_input -= q * gppr0
        return grad_input


entmax15 = lambda input, dim=-1: Entmax15Function.apply(input, dim)
entmoid15 = Entmoid15.apply


class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, *args, **kwargs):
        return self.func(*args, **kwargs)


class ModuleWithInit(nn.Module):
    """ Base class for pytorch module with data-aware initializer on first batch """
    def __init__(self):
        super().__init__()
        self._is_initialized_tensor = nn.Parameter(torch.tensor(0, dtype=torch.uint8), requires_grad=False)
        self._is_initialized_bool = None
        # Note: this module uses a separate flag self._is_initialized so as to achieve both
        # * persistence: is_initialized is saved alongside model in state_dict
        # * speed: model doesn't need to cache
        # please DO NOT use these flags in child modules

    def initialize(self, *args, **kwargs):
        """ initialize module tensors using first batch of data """
        raise NotImplementedError("Please implement ")

    def __call__(self, *args, **kwargs):
        if self._is_initialized_bool is None:
            self._is_initialized_bool = bool(self._is_initialized_tensor.item())
        if not self._is_initialized_bool:
            self.initialize(*args, **kwargs)
            self._is_initialized_tensor.data[...] = 1
            self._is_initialized_bool = True
        return super().__call__(*args, **kwargs)

def download(url, filename, delete_if_interrupted=True, chunk_size=4096):
    """ saves file from url to filename with a fancy progressbar """
    try:
        with open(filename, "wb") as f:
            print("Downloading {} > {}".format(url, filename))
            response = requests.get(url, stream=True)
            total_length = response.headers.get('content-length')

            if total_length is None:  # no content length header
                f.write(response.content)
            else:
                total_length = int(total_length)
                with tqdm(total=total_length) as progressbar:
                    for data in response.iter_content(chunk_size=chunk_size):
                        if data:  # filter-out keep-alive chunks
                            f.write(data)
                            progressbar.update(len(data))
    except Exception as e:
        if delete_if_interrupted:
            print("Removing incomplete download {}.".format(filename))
            os.remove(filename)
        raise e
    return filename


def iterate_minibatches(*tensors, batch_size, shuffle=True, epochs=1,
                        allow_incomplete=True, callback=lambda x:x):
    indices = np.arange(len(tensors[0]))
    upper_bound = int((np.ceil if allow_incomplete else np.floor) (len(indices) / batch_size)) * batch_size
    epoch = 0
    while True:
        if shuffle:
            np.random.shuffle(indices)
        for batch_start in callback(range(0, upper_bound, batch_size)):
            batch_ix = indices[batch_start: batch_start + batch_size]
            batch = [tensor[batch_ix] for tensor in tensors]
            yield batch if len(tensors) > 1 else batch[0]
        epoch += 1
        if epoch >= epochs:
            break


def process_in_chunks(function, *args, batch_size, out=None, **kwargs):
    """
    Computes output by applying batch-parallel function to large data tensor in chunks
    :param function: a function(*[x[indices, ...] for x in args]) -> out[indices, ...]
    :param args: one or many tensors, each [num_instances, ...]
    :param batch_size: maximum chunk size processed in one go
    :param out: memory buffer for out, defaults to torch.zeros of appropriate size and type
    :returns: function(data), computed in a memory-efficient way
    """
    total_size = args[0].shape[0]
    first_output = function(*[x[0: batch_size] for x in args])
    output_shape = (total_size,) + tuple(first_output.shape[1:])
    if out is None:
        out = torch.zeros(*output_shape, dtype=first_output.dtype, device=first_output.device,
                          layout=first_output.layout, **kwargs)

    out[0: batch_size] = first_output
    for i in range(batch_size, total_size, batch_size):
        batch_ix = slice(i, min(i + batch_size, total_size))
        out[batch_ix] = function(*[x[batch_ix] for x in args])
    return out


def check_numpy(x):
    """ Makes sure x is a numpy array """
    if isinstance(x, torch.Tensor):
        x = x.detach().cpu().numpy()
    x = np.asarray(x)
    assert isinstance(x, np.ndarray)
    return x


@contextlib.contextmanager
def nop_ctx():
    yield None


def get_latest_file(pattern):
    list_of_files = glob.glob(pattern) # * means all if need specific format then *.csv
    assert len(list_of_files) > 0, "No files found: " + pattern
    return max(list_of_files, key=os.path.getctime)


def md5sum(fname):
    """ Computes mdp checksum of a file """
    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()


def free_memory(sleep_time=0.1):
    """ Black magic function to free torch memory and some jupyter whims """
    gc.collect()
    torch.cuda.synchronize()
    gc.collect()
    torch.cuda.empty_cache()
    time.sleep(sleep_time)

def to_float_str(element):
    try:
        return str(float(element))
    except ValueError:
        return element
    
class ODST(ModuleWithInit):
    def __init__(self, in_features, num_trees, depth=6, tree_dim=1, flatten_output=True,
                 choice_function=sparsemax, bin_function=sparsemoid,
                 initialize_response_=nn.init.normal_, initialize_selection_logits_=nn.init.uniform_,
                 threshold_init_beta=1.0, threshold_init_cutoff=1.0,
                 ):
        """
        Oblivious Differentiable Sparsemax Trees. http://tinyurl.com/odst-readmore
        One can drop (sic!) this module anywhere instead of nn.Linear
        :param in_features: number of features in the input tensor
        :param num_trees: number of trees in this layer
        :param tree_dim: number of response channels in the response of individual tree
        :param depth: number of splits in every tree
        :param flatten_output: if False, returns [..., num_trees, tree_dim],
            by default returns [..., num_trees * tree_dim]
        :param choice_function: f(tensor, dim) -> R_simplex computes feature weights s.t. f(tensor, dim).sum(dim) == 1
        :param bin_function: f(tensor) -> R[0, 1], computes tree leaf weights
        :param initialize_response_: in-place initializer for tree output tensor
        :param initialize_selection_logits_: in-place initializer for logits that select features for the tree
        both thresholds and scales are initialized with data-aware init (or .load_state_dict)
        :param threshold_init_beta: initializes threshold to a q-th quantile of data points
            where q ~ Beta(:threshold_init_beta:, :threshold_init_beta:)
            If this param is set to 1, initial thresholds will have the same distribution as data points
            If greater than 1 (e.g. 10), thresholds will be closer to median data value
            If less than 1 (e.g. 0.1), thresholds will approach min/max data values.
        :param threshold_init_cutoff: threshold log-temperatures initializer, \in (0, inf)
            By default(1.0), log-remperatures are initialized in such a way that all bin selectors
            end up in the linear region of sparse-sigmoid. The temperatures are then scaled by this parameter.
            Setting this value > 1.0 will result in some margin between data points and sparse-sigmoid cutoff value
            Setting this value < 1.0 will cause (1 - value) part of data points to end up in flat sparse-sigmoid region
            For instance, threshold_init_cutoff = 0.9 will set 10% points equal to 0.0 or 1.0
            Setting this value > 1.0 will result in a margin between data points and sparse-sigmoid cutoff value
            All points will be between (0.5 - 0.5 / threshold_init_cutoff) and (0.5 + 0.5 / threshold_init_cutoff)
        """
        super().__init__()
        self.depth, self.num_trees, self.tree_dim, self.flatten_output = depth, num_trees, tree_dim, flatten_output
        self.choice_function, self.bin_function = choice_function, bin_function
        self.threshold_init_beta, self.threshold_init_cutoff = threshold_init_beta, threshold_init_cutoff

        self.response = nn.Parameter(torch.zeros([num_trees, tree_dim, 2 ** depth]), requires_grad=True)
        initialize_response_(self.response)

        self.feature_selection_logits = nn.Parameter(
            torch.zeros([in_features, num_trees, depth]), requires_grad=True
        )
        initialize_selection_logits_(self.feature_selection_logits)

        self.feature_thresholds = nn.Parameter(
            torch.full([num_trees, depth], float('nan'), dtype=torch.float32), requires_grad=True
        )  # nan values will be initialized on first batch (data-aware init)

        self.log_temperatures = nn.Parameter(
            torch.full([num_trees, depth], float('nan'), dtype=torch.float32), requires_grad=True
        )

        # binary codes for mapping between 1-hot vectors and bin indices
        with torch.no_grad():
            indices = torch.arange(2 ** self.depth)
            offsets = 2 ** torch.arange(self.depth)
            bin_codes = (indices.view(1, -1) // offsets.view(-1, 1) % 2).to(torch.float32)
            bin_codes_1hot = torch.stack([bin_codes, 1.0 - bin_codes], dim=-1)
            self.bin_codes_1hot = nn.Parameter(bin_codes_1hot, requires_grad=False)
            # ^-- [depth, 2 ** depth, 2]

    def forward(self, input):
        assert len(input.shape) >= 2
        if len(input.shape) > 2:
            return self.forward(input.view(-1, input.shape[-1])).view(*input.shape[:-1], -1)
        # new input shape: [batch_size, in_features]

        feature_logits = self.feature_selection_logits
        feature_selectors = self.choice_function(feature_logits, dim=0)
        # ^--[in_features, num_trees, depth]

        feature_values = torch.einsum('bi,ind->bnd', input, feature_selectors)
        # ^--[batch_size, num_trees, depth]

        threshold_logits = (feature_values - self.feature_thresholds) * torch.exp(-self.log_temperatures)

        threshold_logits = torch.stack([-threshold_logits, threshold_logits], dim=-1)
        # ^--[batch_size, num_trees, depth, 2]

        bins = self.bin_function(threshold_logits)
        # ^--[batch_size, num_trees, depth, 2], approximately binary

        bin_matches = torch.einsum('btds,dcs->btdc', bins, self.bin_codes_1hot)
        # ^--[batch_size, num_trees, depth, 2 ** depth]

        response_weights = torch.prod(bin_matches, dim=-2)
        # ^-- [batch_size, num_trees, 2 ** depth]

        response = torch.einsum('bnd,ncd->bnc', response_weights, self.response)
        # ^-- [batch_size, num_trees, tree_dim]

        return response.flatten(1, 2) if self.flatten_output else response

    def initialize(self, input, eps=1e-6):
        # data-aware initializer
        assert len(input.shape) == 2
        if input.shape[0] < 1000:
            warn("Data-aware initialization is performed on less than 1000 data points. This may cause instability."
                 "To avoid potential problems, run this model on a data batch with at least 1000 data samples."
                 "You can do so manually before training. Use with torch.no_grad() for memory efficiency.")
        with torch.no_grad():
            feature_selectors = self.choice_function(self.feature_selection_logits, dim=0)
            # ^--[in_features, num_trees, depth]

            feature_values = torch.einsum('bi,ind->bnd', input, feature_selectors)
            # ^--[batch_size, num_trees, depth]

            # initialize thresholds: sample random percentiles of data
            percentiles_q = 100 * np.random.beta(self.threshold_init_beta, self.threshold_init_beta,
                                                 size=[self.num_trees, self.depth])
            self.feature_thresholds.data[...] = torch.as_tensor(
                list(map(np.percentile, check_numpy(feature_values.flatten(1, 2).t()), percentiles_q.flatten())),
                dtype=feature_values.dtype, device=feature_values.device
            ).view(self.num_trees, self.depth)

            # init temperatures: make sure enough data points are in the linear region of sparse-sigmoid
            temperatures = np.percentile(check_numpy(abs(feature_values - self.feature_thresholds)),
                                         q=100 * min(1.0, self.threshold_init_cutoff), axis=0)

            # if threshold_init_cutoff > 1, scale everything down by it
            temperatures /= max(1.0, self.threshold_init_cutoff)
            self.log_temperatures.data[...] = torch.log(torch.as_tensor(temperatures) + eps)

    def __repr__(self):
        return "{}(in_features={}, num_trees={}, depth={}, tree_dim={}, flatten_output={})".format(
            self.__class__.__name__, self.feature_selection_logits.shape[0],
            self.num_trees, self.depth, self.tree_dim, self.flatten_output
        )
    
class DenseBlock(nn.Sequential):
    def __init__(self, input_dim, layer_dim, num_layers, tree_dim=1, max_features=None,
                 input_dropout=0.0, flatten_output=True, Module=ODST, **kwargs):
        layers = []
        for i in range(num_layers):
            oddt = Module(input_dim, layer_dim, tree_dim=tree_dim, flatten_output=True, **kwargs)
            input_dim = min(input_dim + layer_dim * tree_dim, max_features or float('inf'))
            layers.append(oddt)

        super().__init__(*layers)
        self.num_layers, self.layer_dim, self.tree_dim = num_layers, layer_dim, tree_dim
        self.max_features, self.flatten_output = max_features, flatten_output
        self.input_dropout = input_dropout

    def forward(self, x):
        initial_features = x.shape[-1]
        for layer in self:
            layer_inp = x
            if self.max_features is not None:
                tail_features = min(self.max_features, layer_inp.shape[-1]) - initial_features
                if tail_features != 0:
                    layer_inp = torch.cat([layer_inp[..., :initial_features], layer_inp[..., -tail_features:]], dim=-1)
            if self.training and self.input_dropout:
                layer_inp = F.dropout(layer_inp, self.input_dropout)
            h = layer(layer_inp)
            x = torch.cat([x, h], dim=-1)

        outputs = x[..., initial_features:]
        if not self.flatten_output:
            outputs = outputs.view(*outputs.shape[:-1], self.num_layers * self.layer_dim, self.tree_dim)
        return outputs



In [23]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss



In [24]:

@torch.jit.script
def mish(input):
    '''
    Applies the mish function element-wise:
    mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + exp(x)))
    See additional documentation for mish class.
    '''
    return input * torch.tanh(F.softplus(input))

def swish(x):
    return x * F.sigmoid(x)
class ResLayer(nn.Module):
    def __init__(self, layer):
        super(ResLayer, self).__init__()
        self.layer = layer
    def forward(self, x):
        return x + self.layer(x)

class MoAModel(nn.Module):
    def __init__(self, num_features, num_targets, num_targets_nonscored, hidden_size, dropout=0.4, dropout_input=0.2):
        super(MoAModel, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm1_1 = nn.BatchNorm1d(num_features)
        self.dropout1_1 = nn.Dropout(0.2)
        self.dense1_1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.4)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
                
        self.batch_norm2_1 = nn.BatchNorm1d(hidden_size)
        self.dropout2_1 = nn.Dropout(0.4)
        self.dense2_1 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(0.4)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets_nonscored))
        
        self.dense = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.Dropout(dropout_input),
            nn.utils.weight_norm(nn.Linear(num_features, hidden_size)),
            nn.PReLU(),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(dropout),
            nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size)),
            nn.PReLU(),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(dropout),
        )
        
        self.classifier1 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
        self.classifier2 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets_nonscored))
        #self.dense[-1].bias.data = nn.Parameter(-np.log(torch.from_numpy(y.mean(axis=0)).float()))

        self.embedding1 = nn.Embedding(3,2)
        self.embedding2 = nn.Embedding(2,2)
        
        self.target = 0
    def forward(self, x):
        x_input = x
        #x_emb1 = self.embedding1(x[:, 0].long())
        #x_emb2 = self.embedding1(x[:, 1].long())
        #x = torch.cat([x, x_emb1, x_emb2], dim=1)
        """x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        x_1 = x
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        x_1 = self.output1(x_1)"""
        x = self.dense(x)
        if self.target == 0:
            x = self.classifier1(x)
        else:
            x = self.classifier2(x)
        return x

class MoAModelRes(nn.Module):
    def __init__(self, num_features, num_targets, num_targets_nonscored, hidden_size, dropout=0.4, dropout_input=0.2):
        super(MoAModelRes, self).__init__()
        self.denseres = nn.Sequential(
            nn.BatchNorm1d(num_features),
            nn.Dropout(dropout_input),
            nn.utils.weight_norm(nn.Linear(num_features, hidden_size)),
            ResLayer(nn.Sequential(
                nn.BatchNorm1d(hidden_size),
                nn.PReLU(),
                nn.Dropout(dropout),
                nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size)),
            )),
            ResLayer(nn.Sequential(
                nn.BatchNorm1d(hidden_size),
                nn.PReLU(),
                nn.Dropout(dropout),
                nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size)),
            )),
            ResLayer(nn.Sequential(
                nn.BatchNorm1d(hidden_size),
                nn.PReLU(),
                nn.Dropout(dropout),
                nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size)),
            )),
            ResLayer(nn.Sequential(
                nn.BatchNorm1d(hidden_size),
                nn.PReLU(),
                nn.Dropout(dropout),
                nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size)),
            )),
            nn.PReLU(),
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(dropout),
            #nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
        )
        self.classifier1 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets))
        self.classifier2 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets_nonscored))
    
        self.target = 0
    def forward(self, x):
        x = self.denseres(x)
        if self.target == 0:
            x = self.classifier1(x)
        else:
            x = self.classifier2(x)
        return x

In [25]:
def train_model(model, optimizer, scheduler, criterion, dataloader):
    model.train()
    final_loss = 0
    
    for i, data in enumerate(dataloader):
        #X_batch, y_batch = [t.to(device) for t in data]
        X_batch, y_batch = data
        optimizer.zero_grad()
        
        inputs, targets = X_batch, y_batch
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        
        final_loss += loss.item()
        scheduler.step()
    final_loss /= len(dataloader)
    
    return final_loss


def validate_model(model, criterion, dataloader):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for i, data in enumerate(dataloader):
        #X_batch, y_batch = [t.to(device) for t in data]        
        X_batch, y_batch = data
        
        inputs, targets = X_batch, y_batch
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    final_loss /= len(dataloader)
    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def test_model(model, dataloader):
    model.eval()
    preds = []
    
    for data in tqdm(dataloader):
        inputs = data

        with torch.no_grad():
            outputs = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

In [26]:
from sklearn.metrics import log_loss
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import roc_auc_score, log_loss

class LogitsLogLoss(Metric):
    """
    LogLoss with sigmoid applied
    """

    def __init__(self):
        self._name = "logits_ll"
        self._maximize = False

    def __call__(self, y_true, y_pred):
        """
        Compute LogLoss of predictions.

        Parameters
        ----------
        y_true: np.ndarray
            Target matrix or vector
        y_score: np.ndarray
            Score matrix or vector

        Returns
        -------
            float
            LogLoss of predictions vs targets.
        """
        logits = 1 / (1 + np.exp(-y_pred))
        aux = (1-y_true)*np.log(1-logits+1e-15) + y_true*np.log(logits+1e-15)
        return np.mean(-aux)

In [27]:
def log_loss_numpy(y_true, y_pred):
    loss = 0
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    #y_pred_clip = y_pred
    for i in range(y_pred.shape[1]):
        loss += - np.mean(y_true[:, i] * np.log(y_pred_clip[:, i]) + (1 - y_true[:, i]) * np.log(1 - y_pred_clip[:, i]))
    return loss / y_pred.shape[1]
def sigmoid(logits):
    return 1 / (1 + np.exp(-logits))

In [28]:
import random
from iterstrat.ml_stratifiers import RepeatedMultilabelStratifiedKFold
def test_seed(seed):
    kf = RepeatedMultilabelStratifiedKFold(7, random_state=seed)
    for train_index, valid_index in kf.split(X, y):
        y_train = y[train_index]
        y_train = np.sum(y_train, axis=0)
        if (y_train == 0).any():
            return False
    return True
def get_kfold_seed():
    KFoldSeed = 1

    while True:
        KFoldSeed = random.randint(1,1000000000)
        non_empty = test_seed(KFoldSeed)
        print(KFoldSeed, non_empty)
        if non_empty:
            break
    return KFoldSeed
#KFoldSeed = get_kfold_seed()

In [29]:
from scipy import optimize
from scipy import special

def ensemble_with_weights(weights, y_preds):
    return np.sum([y_preds[i] * weights[i] for i in range(len(weights))], axis=0)

def get_best_weights(y_true, y_preds):
    y_preds = np.array(y_preds, dtype='float64')
    def optimize_func(weights):
        return log_loss_numpy(y_true, ensemble_with_weights(weights, y_preds))
    def optimize_func_two(x):
        return optimize_func([x, 1-x])
    
    #if len(y_preds) == 2:
    #    optimized = optimize.minimize_scalar(optimize_func_two, bounds=(0,1))
    #    print(optimized)
    #    return [optimized.x, 1-optimized.x]
    
    x0 = [1/len(y_preds)] * len(y_preds)
    #x0 = [2/3, 1/3]
    bounds = [(0,1)] * len(y_preds)
    def constr_eq(weights):
        return np.sum(weights) - 1
    constraints = [{'type': 'eq', 'fun': lambda x:1-sum(x)}]
    
    optimized = optimize.minimize(optimize_func, x0, options={'ftol':1e-9,'maxiter':1000}, bounds=bounds, constraints=constraints)
    print(optimized)
    return optimized.x
    

#log_loss_numpy(y_valid, (tabnet_preds/2 + best_preds/2))
#log_loss_numpy(y_valid, ensemble_with_weights(get_best_weights(y_valid, [best_preds, tabnet_preds]), [best_preds, tabnet_preds]))
#ensemble_with_weights([0.5, 0.5],[best_preds, tabnet_preds]) - (tabnet_preds/2 + best_preds/2)

In [30]:
import math
def train_nn(model, trainloader, trainnonscoredloader, validloader, epochs, lr):
    
    model.to(device)
    #optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
    optimizer = LookaheadAdam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs * len(trainloader))
    #scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, epochs=epochs, steps_per_epoch=len(trainloader))
    criterion_val = nn.BCEWithLogitsLoss()
    criterion_train = SmoothBCEwLogits(smoothing=0.001)
    best_loss = 1
    best_preds = []
    t = tqdm(range(epochs))
    for epoch in t:
        if epoch < 1:
            model.target = 1
            train_loss = train_model(model, optimizer, scheduler, criterion_train, trainnonscoredloader)
        else:
            model.target = 0
            train_loss = train_model(model, optimizer, scheduler, criterion_train, trainloader)
        model.target = 0
        valid_loss, valid_preds = validate_model(model, criterion_val, validloader)
        t.set_postfix(train_loss=train_loss, valid_loss=valid_loss, lr=scheduler.get_last_lr()[0])
        
        if math.isnan(valid_loss):
            model.load_state_dict(torch.load("/best_model.pth"))
        elif valid_loss < best_loss:
            best_loss = valid_loss
            best_preds = valid_preds
            torch.save(model.state_dict(), "/best_model.pth")
    
    model.load_state_dict(torch.load("/best_model.pth"))
    
    return model, best_loss, best_preds


In [31]:
#https://www.kaggle.com/optimo/tabnetregressor-2-0-train-infer/data?
TABNET_MAX_EPOCH=200
tabnet_params = dict(n_d=32, n_a=32, n_steps=1, gamma=1.3,
                     lambda_sparse=0, optimizer_fn=LookaheadAdam, #torch.optim.Adam, #
                     optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                     cat_idxs=[0,1],
                     mask_type='entmax',
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.2,),
                     #scheduler_params=dict(gamma=0.9, step_size=5),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     #scheduler_fn=torch.optim.lr_scheduler.StepLR,
                     verbose=10,
                     )

In [32]:

import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [33]:
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
def train_xgb(X_train, y_train, X_valid, y_valid, X_test):
    #xg = []
    xg_pred = np.zeros_like(y_valid, dtype="float64")
    xg_testpred = []
    for i in tqdm(range(206)):
        try:
            xgb = XGBClassifier(tree_method="gpu_hist", n_estimators=1000, objective="binary:logistic")
            xgb.fit(X_train, y_train[:,i], verbose=100, early_stopping_rounds=100, eval_metric="logloss", eval_set=[(X_valid, y_valid[:,i])])
            #xg.append(xgb)
            xg_pred[:,i] = xg[i].predict_proba(X_valid)[:,1]
            xg_testpred.append(xg[i].predict_proba(X_test)[:,1])
        except:
            xg_pred[:,i] = np.mean(y_train[:,i])
            xg_testpred.append([np.mean(y_train[:,i])] * len(X_test))
    
    return xg_pred, np.array(xg_testpred).T

def train_cbc(X_train, y_train, X_valid, y_valid, X_test):
    cbcs = []
    for i in tqdm(range(206)):
        cbc = CatBoostClassifier(task_type="GPU", n_estimators=1000, verbose=100, loss_function="MultiClass", eval_metric="MultiClass", early_stopping_rounds=100)
        cbc.fit(X_train, y_train[:,i], eval_set=[(X_valid, y_valid[:,i])])
        cbcs.append(cbc)
        
        
    cbc_pred = np.zeros_like(y_valid, dtype="float64")
    for i in tqdm(range(206)):
        cbc_pred[:,i] = cbcs[i].predict_proba(X_valid)[:,1]
    
    
    cbc_testpred = []
    for i in tqdm(range(206)):
        cbc_testpred.append(cbc[i].predict_proba(X_test)[:,1])
    
    return cbc_pred, np.array(cbc_testpred).T

In [34]:
epochs = 30
batch_size = 128

In [35]:
models = []
all_submodels = []
losses = []
base_losses = []
masks = []
y_preds = []
tqdm.get_lock().locks = []
KFoldSeed=593306219
for i in range(1):
    kf = MultilabelStratifiedKFold(n_splits=7, shuffle=True, random_state=KFoldSeed)
    for train_index, valid_index in kf.split(X, y):
        X_train, y_train, y_train_nonscored = X[train_index], y[train_index], y_nonscored[train_index]
        X_valid, y_valid, y_valid_nonscored = X[valid_index], y[valid_index], y_nonscored[valid_index]

        train_dataset = MoADataset(X_train, y_train)
        train_nonscored_dataset = MoADataset(X_train, y_train_nonscored)
        valid_dataset = MoADataset(X_valid, y_valid)
        valid_nonscored_dataset = MoADataset(X_valid, y_valid_nonscored)

        trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        trainnonscoredloader = torch.utils.data.DataLoader(train_nonscored_dataset, batch_size=batch_size, shuffle=True)
        validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
        validnonscoredloader = torch.utils.data.DataLoader(valid_nonscored_dataset, batch_size=batch_size, shuffle=False)

        model = MoAModel(
            num_features=len(X_train[0]),
            num_targets=len(y_train[0]),
            num_targets_nonscored=len(y_train_nonscored[0]),
            hidden_size=1024,
            dropout=0.45,
            dropout_input=0.25,
        )

        model.to(device)

        #optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
        optimizer = LookaheadAdam(model.parameters(), lr=0.01, weight_decay=1e-5)
        #optimizer = AdaBelief(model.parameters(), lr=0.001, weight_decay=1e-5, eps=1e-15)
        #optimizer = Lookahead(optimizer)
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs * len(trainloader))
        #scheduler2 = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs * len(trainloader))
        #scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, epochs=epochs, steps_per_epoch=len(trainloader))
        criterion_train = SmoothBCEwLogits(smoothing=0.001)
        criterion_val = nn.BCEWithLogitsLoss()

        best_loss = 1
        best_preds = []
        t = tqdm(range(epochs))
        for epoch in t:
            if epoch < 1:
                model.target = 1
                train_loss = train_model(model, optimizer, scheduler, criterion_train, trainnonscoredloader)
            else:
                model.target = 0
                train_loss = train_model(model, optimizer, scheduler, criterion_train, trainloader)
            model.target = 0
            valid_loss, valid_preds = validate_model(model, criterion_val, validloader)
            t.set_postfix(train_loss=train_loss, valid_loss=valid_loss)

            if valid_loss < best_loss:
                best_loss = valid_loss
                best_preds = valid_preds
                torch.save(model.state_dict(), "/best_model.pth")

        model.target = 0
        model.load_state_dict(torch.load("/best_model.pth"))
        
        
        
        tabnetmodel = TabNetRegressor(**tabnet_params)

        tabnetmodel.fit(X_train=X_train,
                  y_train=y_train,
                  eval_set=[(X_valid, y_valid)],
                  eval_name = ["val"],
                  eval_metric = ["logits_ll"],
                  max_epochs=TABNET_MAX_EPOCH,
                  patience=20, batch_size=256, virtual_batch_size=32,
                  num_workers=1, drop_last=False,
                  # use binary cross entropy as this is not a regression problem
                  #loss_fn=torch.nn.functional.binary_cross_entropy_with_logits)
                  loss_fn=SmoothBCEwLogits(smoothing=0.001))


        tabnet_preds = sigmoid(tabnetmodel.predict(X_valid))
        
        
        xgb_preds, pred3 = train_xgb(X_train, y_train, X_valid, y_valid, X_test)
        
        weights = get_best_weights(y_valid, [best_preds, tabnet_preds, xgb_preds])

        weighted_loss = log_loss_numpy(y_valid, ensemble_with_weights(weights, [best_preds, tabnet_preds, xgb_preds]))
        losses.append(weighted_loss)
        print(best_loss, log_loss_numpy(y_valid, best_preds), log_loss_numpy(y_valid, tabnet_preds), log_loss_numpy(y_valid, xgb_preds), weighted_loss)

        rocauc = roc_auc_score(y_valid.ravel(), ensemble_with_weights(weights, [best_preds, tabnet_preds, xgb_preds]).ravel())
        print("auc", rocauc)
        
        testdataset = MoATestDataset(X_test)
        testloader = torch.utils.data.DataLoader(testdataset, batch_size=batch_size, shuffle=False)

        pred1 = test_model(model, testloader)
        pred2 = sigmoid(tabnetmodel.predict(X_test))
        
        y_preds.append(ensemble_with_weights(weights, [pred1, pred2, pred3]))
        
    #KFoldSeed = get_kfold_seed()

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=593306219 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
  0%|          | 0/30 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629427478/work/torch/csrc/utils/python_arg_parser.cpp:766.)
100%|██████████| 30/30 [00:36<00:00,  1.21s/it, train_loss=0.0178, valid_loss=0.0159]

Device used : cuda


epoch 0  | loss: 0.13261 | val_logits_ll: 0.02281 |  0:00:02s
epoch 10 | loss: 0.02122 | val_logits_ll: 0.01866 |  0:00:26s
epoch 20 | loss: 0.02067 | val_logits_ll: 0.01754 |  0:00:50s
epoch 30 | loss: 0.02054 | val_logits_ll: 0.01774 |  0:01:15s
epoch 40 | loss: 0.02028 | val_logits_ll: 0.01755 |  0:01:40s
epoch 50 | loss: 0.01909 | val_logits_ll: 0.01675 |  0:02:04s
epoch 60 | loss: 0.01821 | val_logits_ll: 0.01682 |  0:02:29s
epoch 70 | loss: 0.0178  | val_logits_ll: 0.01688 |  0:02:52s

Early stopping occured at epoch 70 with best_epoch = 50 and best_val_logits_ll = 0.01675
Best weights from best epoch are automatically used!


  0%|          | 0/206 [00:00<?, ?it/s]

[0]	validation_0-logloss:0.43825
Will train until validation_0-logloss hasn't improved in 100 rounds.
[100]	validation_0-logloss:0.00622
Stopping. Best iteration:
[22]	validation_0-logloss:0.00546



...

[0]	validation_0-logloss:0.43888
Will train until validation_0-logloss hasn't improved in 100 rounds.
[100]	validation_0-logloss:0.01270
Stopping. Best iteration:
[21]	validation_0-logloss:0.00944



100%|██████████| 206/206 [05:05<00:00,  1.48s/it]


     fun: 0.016012968116556645
     jac: array([0.00073224, 0.00073247, 0.00073508])
 message: 'Optimization terminated successfully.'
    nfev: 88
     nit: 17
    njev: 17
  status: 0
 success: True
       x: array([0.93517856, 0.06328445, 0.001537  ])
0.016004465259611605 0.01602654153920367 0.01678688465894295 0.022191324161956058 0.0160129681407004


100%|██████████| 32/32 [00:00<00:00, 330.61it/s]

auc 0.9015704582295588


In [36]:
print(losses)

[0.01597369673662805, 0.0156582414138916, 0.015604031710550696, 0.015866151556941252, 0.0157810972955681, 0.016290326831349598, 0.0160129681407004]


In [37]:
y_test = np.mean(y_preds, axis=0)

In [38]:
df_sample_submission[:] = y_test
df_sample_submission[df_test_features['cp_type'] == 1] = 0
df_sample_submission

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_0004d9e33,0.001093,0.001776,0.002719,0.016622,0.021770,0.005572,0.003239,0.008359,0.000647,0.013381,...,0.001213,0.001930,0.004248,0.001601,0.001472,0.001076,0.001094,0.002537,0.003326,0.002321
id_001897cda,0.000599,0.001111,0.002385,0.004462,0.003290,0.002194,0.005358,0.012118,0.005375,0.012244,...,0.001306,0.001633,0.005218,0.000902,0.007914,0.000814,0.008555,0.001801,0.001484,0.003117
id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
id_00276f245,0.001151,0.001355,0.002322,0.009738,0.013213,0.004700,0.002632,0.004648,0.000462,0.006721,...,0.001045,0.001910,0.003272,0.011773,0.005720,0.001041,0.003496,0.002582,0.000966,0.002597
id_0027f1083,0.002365,0.001874,0.001917,0.017539,0.024573,0.005464,0.003939,0.003128,0.000553,0.010983,...,0.001084,0.000943,0.003168,0.002216,0.001555,0.001077,0.001100,0.002231,0.000536,0.002258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
id_ff7004b87,0.001076,0.001963,0.001686,0.003733,0.009637,0.003538,0.001736,0.003768,0.000550,0.007540,...,0.000893,0.003248,0.002465,0.110462,0.007671,0.001195,0.005320,0.001968,0.000755,0.001555
id_ff925dd0d,0.002672,0.002279,0.001680,0.010633,0.021330,0.005813,0.005144,0.004860,0.000923,0.013452,...,0.000879,0.001602,0.003556,0.001891,0.001885,0.001236,0.001599,0.001951,0.000726,0.001971
id_ffb710450,0.001969,0.001640,0.001389,0.011294,0.030837,0.006395,0.003034,0.005609,0.000462,0.011109,...,0.000739,0.000729,0.002859,0.001119,0.001760,0.000721,0.001015,0.001650,0.000856,0.001576


In [39]:
df_sample_submission.to_csv('submission.csv')